In [1]:
#import libraries
from pymongo import MongoClient
import pandas as pd

In [74]:
def connect_mongo():
    client = MongoClient ('localhost', 27017)
    db = client['companies_clean']
    return db

# Query to MongoDB to get the companies within a radius
def geoquery(lat, long, radius=500):
    db = connect_mongo()
    loc = {"lng": long, "lat": lat}

    result = pd.DataFrame(db.comps.find({
        "coord": {
         "$near": {
           "$geometry": {
              "type": "Point" ,
              "coordinates": [ loc["lng"] , loc["lat"] ]
           },
           "$maxDistance": radius, # In meters 
         }
        }}))
    return result


def get_emp():
    db = connect_mongo()
    all_comps = pd.DataFrame(db.comps.find())
    employees = []
    for i in range(len(all_comps)):
        try:
            comp = geoquery(all_comps['latitude'][i],all_comps['longitude'][i]).sum(axis = 0, skipna = True)
            employees.append(comp['number_of_employees'])
        except:
            employees.append(None)
    return employees


def get_money():
    db = connect_mongo()
    all_comps = pd.DataFrame(db.comps.find())
    money = []
    for i in range(len(all_comps)):
        try:
            comp = geoquery(all_comps['latitude'][i],all_comps['longitude'][i]).sum(axis = 0, skipna = True)
            money.append(comp['total_money_raised'])
        except:
            money.append(None)
    all_comps['money_neighborhood'] = money
    return all_comps

In [75]:
companies = get_money()
companies['emp_neighborhood'] = get_emp()

In [16]:
'''
london = geoquery(51.507410, -0.127685, 5000)
san_francisco = geoquery(37.779248, -122.419243, 5000)
new_york = geoquery(40.725900, -73.981510, 5000)
denver = geoquery(39.739266, -104.987428, 5000)
madrid = geoquery(40.417002, -3.703521, 5000)
locations_list = [london, san_francisco, new_york, denver, madrid]
'''

In [76]:
companies.head()


,_id,category_code,coord,founded_year,latitude,longitude,name,number_of_employees,raised_amount,raised_currency_code,total_money_raised,money_neighborhood,emp_neighborhood
0,5cd5942a666d28a915fcd8d8,games_video,"[-122.409173, 37.788482]",2006,37.788482,-122.409173,Kyte,40.0,2250000.0,USD,23400000.0,476050000.0,365.0
1,5cd5942a666d28a915fcd8d9,advertising,"[-118.1327468, 34.1494713]",2008,34.149471,-118.132747,OpenX,305.0,5000000.0,USD,75500000.0,75500000.0,305.0
2,5cd5942a666d28a915fcd8da,games_video,"[-74.0094471, 40.7464969]",2006,40.746497,-74.009447,Joost,0.0,45000000.0,USD,45000000.0,51700000.0,36.0
3,5cd5942a666d28a915fcd8db,games_video,"[-73.995625, 40.726155]",2007,40.726155,-73.995625,Livestream,120.0,1200000.0,USD,14700000.0,109800000.0,215.0
4,5cd5942a666d28a915fcd8dc,games_video,"[-74.3372, 40.801358]",2007,40.801358,-74.337200,AdaptiveBlue,15.0,1500000.0,USD,24000000.0,24000000.0,15.0
